# Cython Tutorial for Data Scientists

The first thing to do is to load the cython extension

Numba is an option, and supports numpy, but does not support popular packages like Pandas

In [9]:
%load_ext Cython

Functions can be cythonized inline using the cython magic %cython. 

cython compiles the code and stores it as a .so (copiled C function) in the background. In a production environment, this is done manually using a setup.py file, but we will stick with the magic command for now as it is much easier to use.

--annotate produces some output showing the python interaction. Clicking on a line in yellow shows the C code generated for that code block.

In [169]:
%%cython --annotate

cdef int a = 0
for i in range(10):
    a += i
    
print(a)

In [170]:
a

array([[1, 2],
       [3, 4]])

Lets start with a simple example. We first create a function to calculate primes between 0 and kmax

In [46]:
def primes_python(kmax):  # The argument will be converted to int or raise a TypeError.
    n = 0
    k = 0
    i = 0    # These variables are declared with C types.
    p = [0] * 1000  # Non-numpy way to get a list of zeros
    result = []  # A Python type
    if kmax > 1000:
        kmax = 1000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p[k] = n
            k = k + 1
            result.append(n)
        n = n + 1
    return result

primes_python(10)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]

In [41]:
%timeit primes_python(10)

15 µs ± 137 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


So, 15 microseconds per loop. Not bad but it could be much faster. Lets try declaring the important variables as C-types. 

{include a list of useable c-types}

It is important to type all variables. Cython does not give any warnings if you do not, however the code will run much slower.

In [8]:
%%cython

def primes_cython(int kmax):  # The argument will be converted to int or raise a TypeError.
    cdef int n, k, i  # These variables are declared with C types.
    cdef int p[1000]  # Another C type
    result = []  # A Python type
    if kmax > 1000:
        kmax = 1000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p[k] = n
            k = k + 1
            result.append(n)
        n = n + 1
    return result

UsageError: Cell magic `%%cython` not found.


In [50]:
%timeit primes_cython(10)

504 ns ± 3.08 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


The cython implementation runs in ~500 nano-seconds - which is roughly 30 times faster. However, cython improvements vary wildy depending on the applicaion. The average is 60 times improvement, but can reach thousands in some cases.

Since many popular data-science applications such as numpy, scipy, and pandas are written in C (often using cython) cython plays nicely with these toolboxes.

Example from http://nealhughes.net/cython1/

In [2]:
from math import exp
import numpy as np

def genetic_simple(, y, z):

    m = X.shape[0]
    n = X.shape[1]
    Y = np.zeros(N)

    for i in range(N):
        for j in range(N):
            r = 0
            for d in range(D):
                r += (X[j, d] - X[i, d]) ** 2
            r = r**0.5
            Y[i] += beta[j] * exp(-(r * theta)**2)

    return Y

Cython examples with numpy arrays

The NumPy arrays themselves also need to have type information added, and they correspond to the np.ndarray type. The f and g arrays appear as parameters to the function, so the type can be added as part of the function declaration. The h array is initialized to zeros in the code so requires a cdef np.ndarray to be added as follows:

In [38]:
%%cython

# NOTE: recent versions >0.27.3 of cython require both a cimport and import statement
cimport numpy as np 
import numpy as np

def test_ndarray(np.ndarray f, np.ndarray g):
    cdef int fdim1 = f.shape[0]
    cdef int fdim2 = f.shape[1]
    cdef int gdim1 = f.shape[0]
    cdef int gdim2 = f.shape[1]
    cprint("Dimensions are f = (%d x %d), g = (%d x %d)" % (fdim1, fdim2, gdim1, gdim2))

cdef cprint(x):
    print(x)

test_ndarray will take two arguments and output the dimensions
Also, note that cdef functions are only callable from other cython compiled functions.

In [39]:
import numpy as np

test_ndarray(np.ndarray([1,1,1]), np.ndarray([1,1,1]))

Dimensions are f = (1 x 1), g = (1 x 1)


Trying to call cprint directly will throw an error as it is not available outside the compiled module

In [40]:
cprint(h)

NameError: name 'cprint' is not defined

In [77]:
import numpy as np

def dotprod_python(f, g, N):
    h = np.zeros([10, 10], dtype=f.dtype)
    ind = 0
    for m in range(10):
        for n in range(10):
            for i in range(N): # expensive loop
                ind += 1
            h[m,n] = sum(np.reshape(np.dot(f,g),[4,1])) + ind
    return h

In [86]:
%%cython

import numpy as np
cimport numpy as np 

def dotprod_cython(np.ndarray f, np.ndarray g, int N):
    cdef np.ndarray h = np.zeros([10, 10], dtype=f.dtype)
    cdef int ind = 0
    for m in range(10):
        for n in range(10):
            for i in range(N): # expensive loop
                ind += 1
            h[m,n] = sum(np.reshape(np.dot(f,g),[4,1])) + ind
    return h

In [80]:
a = np.array([[1,2],[3,4]])
b = np.array([[11,12],[13,14]])

dotprod_cython(a, b, 10)

array([[ 264,  274,  284,  294,  304,  314,  324,  334,  344,  354],
       [ 364,  374,  384,  394,  404,  414,  424,  434,  444,  454],
       [ 464,  474,  484,  494,  504,  514,  524,  534,  544,  554],
       [ 564,  574,  584,  594,  604,  614,  624,  634,  644,  654],
       [ 664,  674,  684,  694,  704,  714,  724,  734,  744,  754],
       [ 764,  774,  784,  794,  804,  814,  824,  834,  844,  854],
       [ 864,  874,  884,  894,  904,  914,  924,  934,  944,  954],
       [ 964,  974,  984,  994, 1004, 1014, 1024, 1034, 1044, 1054],
       [1064, 1074, 1084, 1094, 1104, 1114, 1124, 1134, 1144, 1154],
       [1164, 1174, 1184, 1194, 1204, 1214, 1224, 1234, 1244, 1254]])

In [88]:
%timeit dotprod_python(a, b, 1)
%timeit dotprod_cython(a, b, 1)

%timeit dotprod_python(a, b, 10000)
%timeit dotprod_cython(a, b, 10000)

%timeit dotprod_python(a, b, 1000000)
%timeit dotprod_cython(a, b, 1000000)

1.21 ms ± 26.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.09 ms ± 87.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
75.4 ms ± 1.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.14 ms ± 262 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
7.7 s ± 728 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.03 ms ± 44.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Try going back and changing the function definition line of the cython file to remove the type declaration on N. (i.e. change "int N" to "N"). Does this significantly affect speed?

[[-0.0296051   3.47832507 -1.26463036 -1.88760799  0.5775514   2.04238108]
 [ 0.93820721  3.02114955 -3.13046424 -2.349613    0.27829437  1.30165722]
 [-2.2548805   3.37134468  3.74490802 -1.59246675 -3.75942332  1.65339708]
 [-0.11647195  3.56313751 -1.91170548  2.69289608  1.17534573 -1.28260992]
 [ 0.66727464 -1.73571576 -2.34210931 -1.83140642 -3.26633701 -2.1832345 ]
 [-1.55244838  0.85376765 -3.64644579  2.86468921 -2.67616455 -2.17009428]
 [-3.95613981  3.61353114 -3.66182083  2.16042169 -2.54160259  0.51652453]
 [-3.96364241 -3.0007559   0.65224998  1.02733306 -2.52220015  2.68426565]]
Generation :  0


AttributeError: module 'ga' has no attribute 'cal_pop_fitness'

In [132]:
import numpy as np

def convolve_python(f, g):
    # f is an image and is indexed by (v, w)
    # g is a filter kernel and is indexed by (s, t),
    #   it needs odd dimensions
    # h is the output image and is indexed by (x, y),
    #   it is not cropped
    if g.shape[0] % 2 != 1 or g.shape[1] % 2 != 1:
        raise ValueError("Only odd dimensions on filter supported")
    # smid and tmid are number of pixels between the center pixel
    # and the edge, ie for a 5x5 filter they will be 2.
    #
    # The output size is calculated by adding smid, tmid to each
    # side of the dimensions of the input image.
    vmax = f.shape[0]
    wmax = f.shape[1]
    smax = g.shape[0]
    tmax = g.shape[1]
    smid = smax // 2
    tmid = tmax // 2
    xmax = vmax + 2 * smid
    ymax = wmax + 2 * tmid
    # Allocate result image.
    h = np.zeros([xmax, ymax], dtype=f.dtype)
    # Do convolution
    for x in range(xmax):
        for y in range(ymax):
            # Calculate pixel value for h at (x,y). Sum one component
            # for each pixel (s, t) of the filter g.
            s_from = max(smid - x, -smid)
            s_to = min((xmax - x) - smid, smid + 1)
            t_from = max(tmid - y, -tmid)
            t_to = min((ymax - y) - tmid, tmid + 1)
            value = 0
            for s in range(s_from, s_to):
                for t in range(t_from, t_to):
                    v = x - smid + s
                    w = y - tmid + t
                    value += g[smid - s, tmid - t] * f[v, w]
            h[x, y] = value
    return h

In [133]:
%prun convolve_python( \
    np.array(np.zeros([100,100])+1, dtype=np.int), \
    np.array(np.zeros([11,11])+1, dtype=np.int))

         48409 function calls in 1.142 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.120    1.120    1.141    1.141 <ipython-input-132-393997e02657>:3(convolve_python)
    24200    0.015    0.000    0.015    0.000 {built-in method builtins.max}
    24200    0.006    0.000    0.006    0.000 {built-in method builtins.min}
        1    0.000    0.000    1.141    1.141 <string>:1(<module>)
        3    0.000    0.000    0.000    0.000 {built-in method numpy.zeros}
        1    0.000    0.000    1.142    1.142 {built-in method builtins.exec}
        2    0.000    0.000    0.000    0.000 {built-in method numpy.array}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

In [150]:
%%cython

import numpy as np
cimport numpy as np

def convolve_cython(np.ndarray f, np.ndarray g):
    cdef int vmax = f.shape[0]
    cdef int wmax = f.shape[1]
    cdef int smax = g.shape[0]
    cdef int tmax = g.shape[1]
    cdef int smid = smax // 2
    cdef int tmid = tmax // 2
    cdef int xmax = vmax + 2 * smid
    cdef int ymax = wmax + 2 * tmid
    cdef int x, y, s_from, s_to, t_from, t_to, value, s, t, v, w
    
    if g.shape[0] % 2 != 1 or g.shape[1] % 2 != 1:
        raise ValueError("Only odd dimensions on filter supported")
    
    # Allocate result image.
    cdef np.ndarray h = np.zeros([xmax, ymax], dtype=f.dtype)
    
    # Do convolution
    for x in range(xmax):
        for y in range(ymax):
            s_from = max(smid - x, -smid)
            s_to = min((xmax - x) - smid, smid + 1)
            t_from = max(tmid - y, -tmid)
            t_to = min((ymax - y) - tmid, tmid + 1)
            value = 0
            for s in range(s_from, s_to):
                for t in range(t_from, t_to):
                    v = x - smid + s
                    w = y - tmid + t
                    value += g[smid - s, tmid - t] * f[v, w]
            h[x, y] = value
    return h

In [149]:
%timeit convolve_cython( \
    np.array(np.zeros([100,100])+1, dtype=np.int), \
    np.array(np.zeros([11,11])+1, dtype=np.int))

1.02 s ± 80.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [146]:
%%cython

import numpy as np
cimport numpy as np

# We now need to fix a datatype for our arrays. I've used the variable
# DTYPE for this, which is assigned to the usual NumPy runtime
# type info object.
DTYPE = np.int

# "ctypedef" assigns a corresponding compile-time type to DTYPE_t. For
# every type in the numpy module there's a corresponding compile-time
# type with a _t-suffix.
ctypedef np.int_t DTYPE_t

cimport cython
@cython.boundscheck(False) # turn off bounds-checking for entire function
@cython.wraparound(False)  # turn off negative index wrapping for entire function
def convolve_cython_indexed(np.ndarray[DTYPE_t, ndim=2] f, np.ndarray[DTYPE_t, ndim=2] g):
    cdef int vmax = f.shape[0]
    cdef int wmax = f.shape[1]
    cdef int smax = g.shape[0]
    cdef int tmax = g.shape[1]
    cdef int smid = smax // 2
    cdef int tmid = tmax // 2
    cdef int xmax = vmax + 2 * smid
    cdef int ymax = wmax + 2 * tmid
    cdef int x, y, s_from, s_to, t_from, t_to, s, t, v, w
    
    if g.shape[0] % 2 != 1 or g.shape[1] % 2 != 1:
        raise ValueError("Only odd dimensions on filter supported")
    
    # Allocate result image.
    cdef np.ndarray[DTYPE_t, ndim=2] h = np.zeros([xmax, ymax], dtype=f.dtype)
    
    cdef DTYPE_t value
    # Do convolution
    for x in range(xmax):
        for y in range(ymax):
            s_from = max(smid - x, -smid)
            s_to = min((xmax - x) - smid, smid + 1)
            t_from = max(tmid - y, -tmid)
            t_to = min((ymax - y) - tmid, tmid + 1)
            value = 0
            for s in range(s_from, s_to):
                for t in range(t_from, t_to):
                    v = x - smid + s
                    w = y - tmid + t
                    value += g[smid - s, tmid - t] * f[v, w]
            h[x, y] = value
    return h

In [148]:
%timeit convolve_cython_indexed( \
    np.array(np.zeros([100,100])+1, dtype=np.int), \
    np.array(np.zeros([11,11])+1, dtype=np.int))

2.17 ms ± 125 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Using Cython with Pandas

In [153]:
import pandas as pd

df = pd.DataFrame({'a': np.random.randn(1000),
                   'b': np.random.randn(1000),
                   'N': np.random.randint(100, 1000, (1000)),
                   'x': 'x'})

def f(x):
    return x * (x - 1)

def integrate_f(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

In [154]:
%timeit df.apply(lambda x: integrate_f(x['a'], x['b'], x['N']), axis=1)

180 ms ± 11.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [155]:
%prun -l 4 df.apply(lambda x: integrate_f(x['a'], x['b'], x['N']), axis=1)

         677162 function calls (672136 primitive calls) in 0.350 seconds

   Ordered by: internal time
   List reduced from 217 to 4 due to restriction <4>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1000    0.174    0.000    0.249    0.000 <ipython-input-153-7dbf247edfac>:11(integrate_f)
   557257    0.075    0.000    0.075    0.000 <ipython-input-153-7dbf247edfac>:8(f)
     3000    0.011    0.000    0.064    0.000 base.py:4702(get_value)
        1    0.008    0.008    0.347    0.347 {pandas._libs.reduction.reduce}

In [166]:
%%cython

def f_plain(x):
    return x * (x - 1)

def integrate_f_plain(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f_plain(a + i * dx)
    return s * dx

In [191]:
%timeit df.apply(lambda x: integrate_f_plain(x['a'], x['b'], x['N']), axis=1)

99.6 ms ± 9.61 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [208]:
%%cython
import cython 

def f_typed(double x):
    return x * (x - 1)

def integrate_f_typed(double a, double b, int N):
    cdef double dx, s
    cdef int i
    dx = (b - a) / N
    s = 0
    for i in range(N):
        s += f_typed(a + i * dx)
    return s * dx

In [209]:
%timeit df.apply(lambda x: integrate_f_typed(x['a'], x['b'], x['N']), axis=1)

59.7 ms ± 3.04 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [195]:
%prun -l 4 df.apply(lambda x: integrate_f_typed(x['a'], x['b'], x['N']), axis=1)

         119905 function calls (114879 primitive calls) in 0.112 seconds

   Ordered by: internal time
   List reduced from 216 to 4 due to restriction <4>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1000    0.025    0.000    0.025    0.000 {_cython_magic_36546310e1dec3aa8c784efa6fe58ae4.integrate_f_typed}
     3000    0.010    0.000    0.055    0.000 base.py:4702(get_value)
     6001    0.006    0.000    0.015    0.000 {pandas._libs.lib.values_from_object}
     3000    0.005    0.000    0.062    0.000 series.py:1068(__getitem__)

In [205]:
%%cython

cimport numpy as np
import numpy as np

def double f_typed(double x): # add exception if something goes wrong
    return x * (x - 1)

def double integrate_f_typed(double a, double b, int N):
    cdef int i
    cdef double s, dx
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f_typed(a + i * dx)
    return s * dx

def np.ndarray[double] apply_integrate_f(np.ndarray col_a, np.ndarray col_b,
                                           np.ndarray col_N):
    assert (col_a.dtype == np.float
            and col_b.dtype == np.float and col_N.dtype == np.int)
    
    cdef Py_ssize_t i, n = len(col_N)
    
    assert (len(col_a) == len(col_b) == n)
    
    cdef np.ndarray[double] res = np.empty(n)
    
    for i in range(len(col_a)):
        res[i] = integrate_f_typed(col_a[i], col_b[i], col_N[i])
        
    return res


Error compiling Cython file:
------------------------------------------------------------
...

cimport numpy as np
import numpy as np

def double f_typed(double x): # add exception if something goes wrong
          ^
------------------------------------------------------------

/Users/darrenprice/.ipython/cython/_cython_magic_719e606c8b75feaa2dd8fb3a4ffb8aff.pyx:5:11: Expected '(', found 'f_typed'. Did you use cdef syntax in a Python declaration? Use decorators and Python type annotations instead.


TypeError: object of type 'NoneType' has no len()

In [203]:
%timeit apply_integrate_f(df['a'].to_numpy(), \
                          df['b'].to_numpy(), \
                          df['N'].to_numpy())

1.39 ms ± 48.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# Dictionaries
We will define a function to loop through a python dictionary.

In [294]:
def dict_python(x):
    s = 0
    for k,v in x.items():
        s = s + v
    return s

In [296]:
x = {'%d' % i:i for i in range(100000)}
%timeit dict_python(x)

7.65 ms ± 139 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


This isn't a good example for speed improvement as writing sum(x.values()) would be much quicker.

In [300]:
%timeit sum(x.values())

1.2 ms ± 30.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [297]:
%%cython

def dict_cython(dict x):
    cdef int s, v
    cdef str k
    s = 0
    for k, v in x.items():
        s = s + v
    return s

In [299]:
%timeit dict_python(x)

7.78 ms ± 682 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [326]:
%%cython

def dict_cython(x, l):
    s = 0
    for litem in l:
        s = s + x[litem]
    return s

In [327]:
k = list(x.keys())
%timeit dict_cython(x, k)

10.8 ms ± 311 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [310]:
def dict_python_iterate(x, l):
    s = 0
    for litem in l:
        s = s + x[litem]
    return s

k = list(x.keys())
dict_python_iterate(x, k)

4999950000

In [321]:
%timeit dict_python_iterate(x, k)

14.4 ms ± 137 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
